In [1]:
import os, json
import requests
import pandas as pd
import numpy as np
import time
import datetime as dt


# launch jupyter from terminal using 
#         jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
# to increase ram allocated (10 GB)
# helps with player aggregating. Doubt 10gbs are needed but hey lol

In [2]:
# starter code for season and previous season ending dates 
# for future automation work




# seasonurl = "http://api.splinterlands.io/settings/"
# season_response = requests.get(seasonurl)
# season_json = season_response.json()

# test_df_3 = pd.json_normalize(season_json)

# season = test_df_3[["season.id", "season.ends", "previous_season.id","previous_season.ends"]]
# season["season.ends"] = pd.to_datetime(season["season.ends"])
# season["previous_season.ends"] = pd.to_datetime(season["previous_season.ends"])

# p_season_end = season["previous_season.ends"].values

# season_end = season["season.ends"].values

In [3]:
# brawl url
brawlurl = "https://api2.splinterlands.com/guilds/brawl_records?cycle="

# api requests for 3 brawls per season
brawl_1 = requests.get(brawlurl + '132').json()
brawl_2 = requests.get(brawlurl + '133').json()
brawl_3 = requests.get(brawlurl + '134').json()

# list of 3 api results
guildsjsonlist = [brawl_1,brawl_2,brawl_3]

# dataframe of normalized api results
guilds_df = pd.json_normalize(guildsjsonlist, record_path = ['results'])

# dataframe with only columns needed for guild report, pre-aggregated
guilds_df_pre_aggregate = guilds_df[['name', 'tournament_id','guild_id','brawl_rank','pts','other_payout','total_merits_payout','wins','losses','draws','brawl_level']]

# aggregated results
aggregate = guilds_df_pre_aggregate.groupby('name').agg({
    'pts': 'sum',
    'brawl_rank': ['mean',(list)],
    'other_payout': 'sum',
    'total_merits_payout': 'sum',
    'wins': 'sum',
    'losses': 'sum',
    'draws': 'sum',
    'brawl_level': 'last',
    'guild_id': 'first'
})

# dataframe of aggregated results
guild_agg_df = pd.DataFrame(aggregate)

# updating column names
guild_pre_sort_bin = guild_agg_df.droplevel(1,axis=1)
guild_pre_sort_bin.rename(columns = {'pts':'Points', 'brawl_rank':'Avg_Finish', 'brawl_rank':'Finishes', 'other_payout':'Crowns',
                           'total_merits_payout':'Merits', 'wins':'W', 'losses':'L', 'draws':'D', 
                                 'brawl_level': 'Arena_Level'}, inplace=True)
guild_pre_sort_bin.columns.values[1] = 'Avg_Finish'

#reset index, fixing 'name' column's name. Haha. 
guild_pre_sort_bin.reset_index(inplace=True)
guild_pre_sort_bin.columns.values[0] = 'Name'

#sort
guild_pre_sort_bin = guild_pre_sort_bin.sort_values('Points', ascending=False)
guild_pre_sort_bin

,Name,Points,Avg_Finish,Finishes,Crowns,Merits,W,L,D,Arena_Level,guild_id
530,Summoner's Tribe,1128,1.000000,"[1, 1, 1]",462,757197,376,97,0,8,fd95485db324a466f96e8d93f2f7e7f5dcb7feb2
302,Legendary Dragons,1050,1.000000,"[1, 1, 1]",420,704403,350,124,0,8,c33e99909c5d4c219ee9ba4f62750d3f83a0bac7
605,The Guild of Neoxian,1035,1.666667,"[1, 2, 2]",412,654003,345,143,0,8,f1840e0d297c7a6aaf4fbb9dd68dd88d2b60af2c
252,Immortal Gods,1032,2.333333,"[2, 1, 4]",407,608391,344,148,0,8,391cc53158d4b06e7e5d8f0dfba614267905b8cf
479,Samurai Soul 🗻 LLAC,975,3.000000,"[3, 3, 3]",387,445998,325,163,0,7,576b2d4ebc76532f7e9236012a7bbaa3ad9dda8f
...,...,...,...,...,...,...,...,...,...,...,...
447,Rebels' Haven,6,8.000000,[8],1,38,2,4,0,1,c71fb1caa1b4deb568c1683e04810b8de982775b
635,Thousand Skies Nomads,6,9.333333,"[9, 9, 10]",1,0,2,21,0,1,c9282a559a396f08098e28cf0b25750b53a93eaf
587,The Black Bulls,0,10.000000,[10],0,0,0,19,0,1,5c3ba10370c26dab2e3cfd38ed2dac569d36eb57
483,Shadow Warriors,0,9.000000,"[8, 10]",2,46,0,42,0,1,9c44522ae395318103a88dbdee894a28e7dbbae9


In [4]:
# Top 25, Tier 1 Brawls

# sorting into tier
guild_df_tier1 = guild_pre_sort_bin[(guild_pre_sort_bin.Arena_Level == 1) | (guild_pre_sort_bin.Arena_Level == 2)]

# drop arena_level & guild_id columns
guild_df_tier1_final = guild_df_tier1.drop(['Arena_Level', 'guild_id'], axis=1)

#creating and printing a table of the report in markdown format
guild_tier1_table = guild_df_tier1_final.head(25).to_markdown(index=False)
print(guild_tier1_table)

| Name                   |   Points |   Avg_Finish | Finishes   |   Crowns |   Merits |   W |   L |   D |
|:-----------------------|---------:|-------------:|:-----------|---------:|---------:|----:|----:|----:|
| POSEIDON'S TRIDENT     |      429 |      1.33333 | [1, 1, 2]  |       53 |    46491 | 143 |  64 |   0 |
| [YGG] Legion II        |      420 |      1       | [1, 1, 1]  |       48 |    37453 | 140 |  56 |   0 |
| SPECIAL OPPS           |      417 |      1       | [1, 1, 1]  |       47 |    35749 | 139 |  39 |   0 |
| The Hammers of Dawn    |      408 |      2       | [2, 1, 3]  |       44 |    31018 | 136 |  89 |   0 |
| LW                     |      402 |      1.33333 | [1, 2, 1]  |       45 |    31983 | 134 |  60 |   0 |
| Master Splinter        |      399 |      1.66667 | [1, 2, 2]  |       46 |    33816 | 133 |  62 |   0 |
| Balasador [PHL]        |      396 |      1.33333 | [2, 1, 1]  |       42 |    27539 | 132 |  41 |   0 |
| 8th ELEMENT            |      393 |      2.6

In [5]:
 #Top 25, Tier 2 Brawls

# sorting into tier
guild_df_tier2 = guild_pre_sort_bin[(guild_pre_sort_bin.Arena_Level == 3) | (guild_pre_sort_bin.Arena_Level == 4)]

# drop arena_level & guild_id columns
guild_df_tier2_final = guild_df_tier2.drop(['Arena_Level', 'guild_id'], axis=1)

#creating and printing a table of the report in markdown format
guild_tier2_table = guild_df_tier2_final.head(25).to_markdown(index=False)
print(guild_tier2_table)

| Name                     |   Points |   Avg_Finish | Finishes   |   Crowns |   Merits |   W |   L |   D |
|:-------------------------|---------:|-------------:|:-----------|---------:|---------:|----:|----:|----:|
| Humen not welcome        |      726 |      1       | [1, 1, 1]  |      135 |   152800 | 242 | 100 |   0 |
| Forbidden Summoners      |      726 |      1       | [1, 1, 1]  |      155 |   176187 | 242 |  89 |   0 |
| Samurai Soul 🌸 LLAC     |      690 |      1.33333 | [1, 2, 1]  |      140 |   209336 | 230 | 109 |   0 |
| Goldcards                |      654 |      1.33333 | [1, 2, 1]  |      138 |   143050 | 218 |  80 |   0 |
| Team Rocket              |      654 |      1       | [1, 1, 1]  |      150 |   164800 | 218 |  88 |   0 |
| Pizza A Slice of Madness |      654 |      2       | [2, 2, 2]  |      130 |   116689 | 218 | 123 |   0 |
| YGG Fighting Illini      |      639 |      1.33333 | [1, 1, 2]  |      132 |   159203 | 213 |  99 |   0 |
| COLLECTIVE VISION        | 

In [6]:
# Top 25, Tier 3 Brawls

# sorting into tier
guild_df_tier3 = guild_pre_sort_bin[(guild_pre_sort_bin.Arena_Level == 5) | (guild_pre_sort_bin.Arena_Level == 6)]

# drop arena_level & guild_id columns
guild_df_tier3_final = guild_df_tier3.drop(['Arena_Level', 'guild_id'], axis=1)

#creating and printing a table of the report in markdown format
guild_tier3_table = guild_df_tier3_final.head(25).to_markdown(index=False)
print(guild_tier3_table)

| Name                        |   Points |   Avg_Finish | Finishes   |   Crowns |   Merits |   W |   L |   D |
|:----------------------------|---------:|-------------:|:-----------|---------:|---------:|----:|----:|----:|
| YGG Brawlers (Niðavellir)   |      900 |      1.66667 | [1, 3, 1]  |      261 |   346014 | 300 | 122 |   0 |
| SPLINTERLANDS.RU [PRO]      |      885 |      1.33333 | [1, 2, 1]  |      255 |   347165 | 295 | 129 |   0 |
| YGG Brawlers (Ásgard)       |      885 |      1.33333 | [1, 2, 1]  |      265 |   394902 | 295 | 101 |   0 |
| Absalom                     |      879 |      2       | [1, 2, 3]  |      254 |   323107 | 293 | 126 |   0 |
| [KoG] Guardians of Glory    |      867 |      1.33333 | [2, 1, 1]  |      269 |   326700 | 289 | 109 |   0 |
| Team Possible               |      867 |      1       | [1, 1, 1]  |      254 |   402540 | 289 | 101 |   0 |
| PeakMonsters Pirates        |      846 |      1.66667 | [2, 1, 2]  |      253 |   269604 | 282 | 124 |   0 |
|

In [7]:
# Top 25, Tier 4 Brawls

# sorting into tier
guild_df_tier4 = guild_pre_sort_bin[guild_pre_sort_bin.Arena_Level > 6]

# drop arena_level & guild_id columns
guild_df_tier4_final = guild_df_tier4.drop(['Arena_Level', 'guild_id'], axis=1)

#creating and printing a table of the report in markdown format
guild_tier4_table = guild_df_tier4_final.head(25).to_markdown(index=False)
print(guild_tier4_table)

| Name                      |   Points |   Avg_Finish | Finishes   |   Crowns |   Merits |   W |   L |   D |
|:--------------------------|---------:|-------------:|:-----------|---------:|---------:|----:|----:|----:|
| Summoner's Tribe          |     1128 |      1       | [1, 1, 1]  |      462 |   757197 | 376 |  97 |   0 |
| Legendary Dragons         |     1050 |      1       | [1, 1, 1]  |      420 |   704403 | 350 | 124 |   0 |
| The Guild of Neoxian      |     1035 |      1.66667 | [1, 2, 2]  |      412 |   654003 | 345 | 143 |   0 |
| Immortal Gods             |     1032 |      2.33333 | [2, 1, 4]  |      407 |   608391 | 344 | 148 |   0 |
| Samurai Soul 🗻 LLAC      |      975 |      3       | [3, 3, 3]  |      387 |   445998 | 325 | 163 |   0 |
| [KoG] Shield of Glory     |      936 |      2.33333 | [2, 4, 1]  |      375 |   498122 | 312 | 173 |   0 |
| Legendary Raptors         |      849 |      2.33333 | [3, 2, 2]  |      341 |   380320 | 283 | 191 |   0 |
| PeakMonsters [PREM

In [8]:
# extract Brawl IDs and Guild IDs from pre-aggregated DataFrame 
brawlids = guilds_df_pre_aggregate['tournament_id'].values
guildids = guilds_df_pre_aggregate['guild_id'].values

#create empty list to hold api data
brawlerdata = []

#create counter for time delay
k = 0

# loop through all ids and make api calls
for i,j in zip(brawlids, guildids):
    
    #initialize counter increments
    k = k + 1
    
    #initialize time delay on conditional
    if (k % 25 == 0 and k >= 25): #prob could just leave k >= 25 out
        time.sleep(10)
    
    # api call
    try:
        brawlerurl = f"https://api2.splinterlands.com/tournaments/find_brawl?id={i}&guild_id={j}"
        brawler_json = requests.get(brawlerurl).json()
    
        #append each call's data to brawlerdata
        brawlerdata.append(brawler_json)
    except:
        pass



In [9]:
# normalize brawlerdata to DataFrame
try:
    brawler_df = pd.json_normalize(brawlerdata, record_path = ['players'], errors='ignore')
except KeyError as e:
    print (f"Unable to normalize json: {json.dumps(brawlerdata, indent=4)}")


In [10]:
brawler_df

,total_battles,entered_battles,player,join_date,wins,losses,draws,finish,meta_pts,meta_pts_float,...,player_data.title_post,player_data.display_name,player_data.league,player_data.modern_league,player_data.collection_power,player_data.starter_pack_purchase,player_data.guild_id,player_data.guild_rank,player_data.guild_join_date,player_data.guild_status
0,4,4,wesquin703,2023-01-28T02:50:57.000Z,4,0,0,6,36,0,...,None,None,0,7,252965,True,41397704a4e74dc79f06b88745488705a36b537a,5.0,2021-09-12T04:34:45.000Z,active
1,5,5,lichdeath,2023-01-28T02:10:48.000Z,4,1,0,9,42,0,...,None,None,0,3,25915,True,41397704a4e74dc79f06b88745488705a36b537a,1.0,2022-12-26T19:35:27.000Z,active
2,7,7,priesters,2023-01-28T15:50:48.000Z,4,3,0,14,54,0,...,None,None,0,7,60650,True,41397704a4e74dc79f06b88745488705a36b537a,1.0,2022-01-20T23:45:30.000Z,active
3,3,3,pat1x,2023-01-28T10:31:06.000Z,3,0,0,15,18,2025,...,None,None,4,7,254825,True,41397704a4e74dc79f06b88745488705a36b537a,2.0,2021-12-01T21:57:48.000Z,active
4,4,4,rickvd91,2023-01-28T11:28:18.000Z,3,1,0,16,30,0,...,None,None,0,6,120615,True,41397704a4e74dc79f06b88745488705a36b537a,1.0,2022-08-02T17:00:57.000Z,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23635,8,8,juliusm518,2023-02-07T03:02:51.000Z,5,3,0,64,72,0,...,None,None,0,9,368840,True,b97797c12230454566917ae62ced0174586707dc,1.0,2022-09-22T02:22:42.000Z,active
23636,8,8,renan.santoliv,2023-02-08T23:12:48.000Z,4,4,0,92,54,2118,...,None,None,4,0,63345,True,b97797c12230454566917ae62ced0174586707dc,1.0,2022-11-26T02:39:18.000Z,active
23637,8,8,horsebox,2023-02-07T10:02:45.000Z,3,5,0,120,42,0,...,None,None,0,10,316891,True,b97797c12230454566917ae62ced0174586707dc,1.0,2021-09-08T18:58:15.000Z,active
23638,8,8,acantoni,2023-02-08T22:42:51.000Z,0,8,0,168,0,2758,...,None,None,9,0,1279417,True,b97797c12230454566917ae62ced0174586707dc,2.0,2021-10-06T10:13:00.000Z,active


In [11]:
# aggregate the needed columns on the 'player' column for report
player_agg = brawler_df.groupby('player').agg({
    'total_battles': 'sum',
    'wins': 'sum',
    'losses': 'sum',
    'draws': 'sum',
    'player_data.guild_id': 'first'
})

# create dataframe from aggregates
player_agg_df = pd.DataFrame(player_agg)


In [12]:
# reset index
player_agg_df.reset_index(inplace=True)

# creating new mathy fields
player_agg_df['Points'] = player_agg_df.wins * 3
player_agg_df['PPB'] = player_agg_df.Points / player_agg_df.total_battles

# merge player & guild aggregated dataframes to bring in necessary guild-level info
player_agg_merge=pd.merge(player_agg_df,guild_pre_sort_bin, left_on='player_data.guild_id', right_on='guild_id')
player_agg_merge

# create new, clean player dataframe with columns 
player_df = player_agg_merge[['player', 'PPB', 'Points_x', 'total_battles', 'Merits', 'wins', 'losses', 'draws', 'Name', 'Arena_Level']]

# rename columns
player_df.rename(columns={'player':'Player','Points_x':'Points','total_battles':'Battles',
                          'wins':'W','losses':'L','draws':'D'},inplace=True)

#sort
player_df = player_df.sort_values('Points', ascending=False)


C:\Users\Andy\AppData\Local\Temp\ipykernel_16556\3666400019.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df.rename(columns={'player':'Player','Points_x':'Points','total_battles':'Battles',


,Player,PPB,Points,Battles,Merits,W,L,D,Name,Arena_Level
3083,amaari,2.880000,72,25,159203,24,1,0,YGG Fighting Illini,3
3612,flavianoramos,2.875000,69,24,326700,23,1,0,[KoG] Guardians of Glory,6
7102,sniper-vc1,2.653846,69,26,58515,23,3,0,Death Squad Cuba,3
2924,byzantinekitty,3.000000,69,23,757197,23,0,0,Summoner's Tribe,8
3774,iamnothing2,2.653846,69,26,269604,23,3,0,PeakMonsters Pirates,6
...,...,...,...,...,...,...,...,...,...,...
3901,aorta,0.000000,0,4,19956,0,4,0,Me-Fi Guild,1
3884,zecoronas,0.000000,0,14,5067,0,14,0,Oasis GUILD | 綠洲公會,1
6013,dufu,0.000000,0,6,690,0,6,0,天王盖地虎,1
3883,wbl1992304,0.000000,0,9,5067,0,9,0,Oasis GUILD | 綠洲公會,1


In [13]:
# Top 25 players, Tier 1 Brawls

# sorting into tier
player_df_tier1 = player_df[(player_df.Arena_Level == 1) | (player_df.Arena_Level == 2)]

# drop arena_level column
player_df_tier1_final = player_df_tier1.drop(['Arena_Level'], axis=1)

# adding '@' to player name to tag them in report post
player_df_tier1_final['Player'] = ('@' + player_df_tier1_final['Player'])

#creating and printing a table of the report in markdown format
player_tier1_table = player_df_tier1_final.head(25).to_markdown(index=False)
print(player_tier1_table)

| Player           |     PPB |   Points |   Battles |   Merits |   W |   L |   D | Name                    |
|:-----------------|--------:|---------:|----------:|---------:|----:|----:|----:|:------------------------|
| @matej-tyty      | 2.55556 |       69 |        27 |     5260 |  23 |   4 |   0 | SK/CZ - Slováci a Češi  |
| @kaitm42         | 2.86957 |       66 |        23 |    16316 |  22 |   1 |   0 | The Family              |
| @noobcake        | 2.86364 |       63 |        22 |    35749 |  21 |   1 |   0 | SPECIAL OPPS            |
| @chirichan       | 2.625   |       63 |        24 |    20760 |  21 |   3 |   0 | Lost Legion             |
| @johnhtims.pal   | 2.6087  |       60 |        23 |    29393 |  20 |   3 |   0 | JHT'S Scholars          |
| @oksanauk        | 2.4     |       60 |        25 |    13337 |  20 |   5 |   0 | NeOn.                   |
| @m1tz            | 2.72727 |       60 |        22 |    29883 |  20 |   2 |   0 | Mushroom Warriors BR    |
| @mario01         

In [14]:
# Top 25 players, Tier 2 Brawls

# sorting into tier
player_df_tier2 = player_df[(player_df.Arena_Level == 3) | (player_df.Arena_Level == 4)]

# drop arena_level column
player_df_tier2_final = player_df_tier2.drop(['Arena_Level'], axis=1)

# adding '@' to player name to tag them in report post
player_df_tier2_final['Player'] = ('@' + player_df_tier2_final['Player'])

#creating and printing a table of the report in markdown format
player_tier2_table = player_df_tier2_final.head(25).to_markdown(index=False)
print(player_tier2_table)

| Player            |     PPB |   Points |   Battles |   Merits |   W |   L |   D | Name                      |
|:------------------|--------:|---------:|----------:|---------:|----:|----:|----:|:--------------------------|
| @amaari           | 2.88    |       72 |        25 |   159203 |  24 |   1 |   0 | YGG Fighting Illini       |
| @sniper-vc1       | 2.65385 |       69 |        26 |    58515 |  23 |   3 |   0 | Death Squad Cuba          |
| @gr8u             | 2.76    |       69 |        25 |   176187 |  23 |   2 |   0 | Forbidden Summoners       |
| @captainloken     | 2.64    |       66 |        25 |   124850 |  22 |   3 |   0 | Legendary Dragon Babies   |
| @wayn3            | 2.86957 |       66 |        23 |   164800 |  22 |   1 |   0 | Team Rocket               |
| @mcrahman91       | 2.64    |       66 |        25 |    95150 |  22 |   3 |   0 | Electronz                 |
| @anime4life       | 2.53846 |       66 |        26 |   176187 |  22 |   4 |   0 | Forbidden Summoners 

In [15]:
# Top 25 players, Tier 3 Brawls

# sorting into tier
player_df_tier3 = player_df[(player_df.Arena_Level == 5) | (player_df.Arena_Level == 6)]

# drop arena_level column
player_df_tier3_final = player_df_tier3.drop(['Arena_Level'], axis=1)

# adding '@' to player name to tag them in report post
player_df_tier3_final['Player'] = ('@' + player_df_tier3_final['Player'])

#creating and printing a table of the report in markdown format
player_tier3_table = player_df_tier3_final.head(25).to_markdown(index=False)
print(player_tier3_table)

| Player         |     PPB |   Points |   Battles |   Merits |   W |   L |   D | Name                      |
|:---------------|--------:|---------:|----------:|---------:|----:|----:|----:|:--------------------------|
| @flavianoramos | 2.875   |       69 |        24 |   326700 |  23 |   1 |   0 | [KoG] Guardians of Glory  |
| @iamnothing2   | 2.65385 |       69 |        26 |   269604 |  23 |   3 |   0 | PeakMonsters Pirates      |
| @bubbaklubba   | 2.65385 |       69 |        26 |   269604 |  23 |   3 |   0 | PeakMonsters Pirates      |
| @saffisara     | 2.75    |       66 |        24 |   130350 |  22 |   2 |   0 | The Alliance              |
| @wileyzile     | 2.53846 |       66 |        26 |   346014 |  22 |   4 |   0 | YGG Brawlers (Niðavellir) |
| @eligere       | 2.53846 |       66 |        26 |   346014 |  22 |   4 |   0 | YGG Brawlers (Niðavellir) |
| @maplelicious  | 2.75    |       66 |        24 |   330548 |  22 |   2 |   0 | Immortal Gods 4           |
| @max1ne        | 

In [16]:
# Top 25 players, Tier 4 Brawls

# sorting into tier
player_df_tier4 = player_df[(player_df.Arena_Level > 6)]

# drop arena_level column
player_df_tier4_final = player_df_tier4.drop(['Arena_Level'], axis=1)

# adding '@' to player name to tag them in report post
player_df_tier4_final['Player'] = ('@' + player_df_tier4_final['Player'])

#creating and printing a table of the report in markdown format
player_tier4_table = player_df_tier4_final.head(25).to_markdown(index=False)
print(player_tier4_table)

| Player            |     PPB |   Points |   Battles |   Merits |   W |   L |   D | Name                    |
|:------------------|--------:|---------:|----------:|---------:|----:|----:|----:|:------------------------|
| @byzantinekitty   | 3       |       69 |        23 |   757197 |  23 |   0 |   0 | Summoner's Tribe        |
| @jayemm3          | 2.75    |       66 |        24 |   498122 |  22 |   2 |   0 | [KoG] Shield of Glory   |
| @bubke            | 2.75    |       66 |        24 |   608391 |  22 |   2 |   0 | Immortal Gods           |
| @byzantinist      | 3       |       66 |        22 |   757197 |  22 |   0 |   0 | Summoner's Tribe        |
| @ign1te           | 2.75    |       66 |        24 |   498122 |  22 |   2 |   0 | [KoG] Shield of Glory   |
| @namelessnameless | 2.625   |       63 |        24 |   654003 |  21 |   3 |   0 | The Guild of Neoxian    |
| @brilliant-idea   | 2.86364 |       63 |        22 |   704403 |  21 |   1 |   0 | Legendary Dragons       |
| @xawi   